In [ ]:
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import glob
import re
from tkinter import filedialog as fd
from tkinter import *

In [ ]:
def getFilePathFromDialog(_initialdir):
    root = Tk()
    FilePath = fd.askopenfilename(initialdir = _initialdir)
    root.destroy()
    return FilePath

def getDirectoryPathFromDialog(_initialdir):
    root = Tk()
    Directory = fd.askdirectory(initialdir = _initialdir)
    root.destroy()
    return Directory

In [ ]:
def readFile(path, skiprows):
    return np.loadtxt(path, dtype=float, skiprows=skiprows)

def readFileFromDialog(skiprows):
    return readFile(getFilePathFromDialog(dataPath),skiprows)

In [ ]:
def readParticleConfig(path,ConfigurationNumber):
    TotalNumberOfParticles = int(re.search(r'(?<=N=)[\d]*',path).group())
    with open(path,'r') as F:
        Data = F.read()
    RowOffset = 1+ConfigurationNumber*(TotalNumberOfParticles+2)
    print("RowOffset: ",RowOffset)
    Data = Data.strip()
    FirstRow = Data.split(sep='\n')[RowOffset]
    NumberOfAParticles = int(FirstRow.split()[5].strip('|'))
    NumberOfBParticles = int(FirstRow.split()[7].strip('|'))
    ListOfRows = Data.split(sep='\n')[RowOffset+1:RowOffset+1+TotalNumberOfParticles]
    if (len(ListOfRows) > 0):
        NumberOfColumns = len(ListOfRows[0].split()) - 1
    else:
        NumberOfColumns = 0
    APositions = np.empty((NumberOfAParticles, NumberOfColumns))
    BPositions = np.empty((NumberOfBParticles, NumberOfColumns))
    AIndex = 0
    BIndex = 0
    
    for i in range(len(ListOfRows)):
        if (ListOfRows[i].split()[NumberOfColumns] == 'A'):
            for j in range(NumberOfColumns):
                APositions[AIndex,j] = ListOfRows[i].split()[j]
            AIndex += 1
        else:
            for j in range(NumberOfColumns):
                BPositions[BIndex,j] = ListOfRows[i].split()[j]
            BIndex += 1
    return APositions, BPositions

def readParticleConfigFromDialog(ConfigurationNumber):
    return readParticleConfig(getFilePathFromDialog(dataPath),ConfigurationNumber)

In [ ]:
def LinearFunction(x,m,n):
    return m*x+n

In [ ]:
dataPath = getDirectoryPathFromDialog('')

## Energy series

In [ ]:
EnergySeries = readFileFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(2,2,squeeze=False,figsize=(15,9))
Ax[0,0].plot(EnergySeries[:1000,0],label='U')
Ax[0,0].plot(EnergySeries[:1000,1],label='T')
Ax[0,0].plot(EnergySeries[:1000,2],label='H=U+T')
Ax[0,0].legend()
Ax[0,1].plot(EnergySeries[:,2],label='H=U+T')
Ax[0,1].set_ylabel('H=T+U')
MeanH = np.mean(EnergySeries[:,2])
print("H=",MeanH,'+-',np.std(EnergySeries[:,2]))
print("U=",np.mean(EnergySeries[:,0]),'+-',np.std(EnergySeries[:,0]))
print("T=",np.mean(EnergySeries[:,1]),'+-',np.std(EnergySeries[:,1]))
Ax[0,1].plot(MeanH*np.ones(len(EnergySeries[:,2])))
n,bins,patches = Ax[1,1].hist(EnergySeries[:,2],bins = 41, density = True)
Ax[1,1].vlines(MeanH,0,np.max(n),color='darkorange')
Ax[1,1].set_title("Histogram of H")
Ax[1,0].plot(EnergySeries[:,1])
Ax[1,0].set_ylabel('T')

In [ ]:
PotEnergySeriesMC = readFileFromDialog(1)

In [ ]:
Fix, Ax = plt.subplots(2,1,squeeze=False,figsize=(10,12))
Ax[0,0].plot(PotEnergySeriesMC)
Ax[0,0].plot(EnergySeries[:,0])
print("U Monte Carlo = ",np.mean(PotEnergySeriesMC),"+-",np.std(PotEnergySeriesMC))
print("U MD = ",np.mean(EnergySeries[:,0]),"+-",np.std(EnergySeries[:,0]))
a = Ax[1,0].hist(PotEnergySeriesMC,bins = 41, density = True, alpha = 0.7)
a = Ax[1,0].hist(EnergySeries[:,0],bins = 41, density = True,alpha =0.7)

## Average displacement

In [ ]:
traveledData = readFileFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(len(traveledData[0]),2,squeeze=False, figsize = (10,5*len(traveledData[0])), dpi = 100)
Means = np.zeros(len(traveledData[0]))
for i in range(len(traveledData[0])):
    Ax[i,0].plot(traveledData[1:,i],'.',label='y={}'.format(traveledData[0,i]))
    Means[i] = np.mean(traveledData[1:,i])
    Ax[i,0].plot(Means[i]*np.ones(len(traveledData[1:,i])))
    Ax[i,0].legend()
    n,bins,patches = Ax[i,1].hist(traveledData[1:,i],bins = 41, density = True)
    Ax[i,1].vlines(Means[i],0,np.max(n),color='darkorange')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False,figsize=(5,5), dpi = 100)
Ax[0,0].plot(Means,traveledData[0],'.')
Ax[0,0].set_xlabel('average $\Delta x / \sigma$ per sweep')
Ax[0,0].set_ylabel('y / $\sigma$')
#Ax[0,0].set_ylim(0,1)
popt, pcov = curve_fit(LinearFunction, Means[:], traveledData[0][:])
print(popt)
print(popt[1]/popt[0])
print('shear rate: {:.3E}'.format(1/popt[0]))
Ax[0,0].plot(Means[:],LinearFunction(Means[:],*popt))
Ax[0,0].set_title('Average $\Delta x$ vs y')
Ax[0,0].ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
#Fig.savefig("plots/shear_analysis/avg_delta_x_field_no_lees_edwards_T=1.png",format='png')